# Hubs Take-home Task Insights

Hello! These are insights about the holes data that I've gathered from the raw data provided for DFM analysis, which is then transformed using the transformation pipeline I designed shown in the photo found in the same directory of this notebook.

In this notebook, the transformed table for the holes data from Pandas DataFrames are ingested into a SQLite database, and I query from this table to gather insights.


### Import required packages and functions

In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine, text
from data_transformation.main import main

### Raw data

First, I extract the data from the provided Parquet file using Pandas's `read_parquet` method.

In [2]:
data = pd.read_parquet("2023 DE_case_dataset.gz.parquet")

In [3]:
data

,created,updated,queued,geometric_heuristics,holes,job_run_time,latheability,machining_directions,multipart,neighbors,poles,sheet_like_shape,unmachinable_edges,extrusion_height,units,status,time,uuid
0,2020-02-15 22:46:41,2020-02-15 22:47:00,2020-02-15 22:46:42,"{""unit"":""milliseconds"",""value"":56}",<NA>,"{""unit"":""milliseconds"",""value"":7491}","{""axis"":{""x"":1,""y"":0,""z"":0},""fraction"":0.84588...","{""box_volume"":{""unit"":""mm3"",""value"":15590.8676...","{""multibody"":false,""patches"":{""count"":1,""not_t...","{""unit"":""mm3"",""value"":[8576,6425,3703,762,24,0...",<NA>,<NA>,"{""count"":1469,""edge_list_url"":""s3://3dhubs-pro...",NaN,mm,completed,7547,92fe59e7-eae0-4ba9-bf74-22a82f2c2c83
1,2020-05-13 14:17:10,2020-05-13 14:17:11,2020-05-13 14:17:10,"{""unit"":""milliseconds"",""value"":44}",<NA>,"{""unit"":""milliseconds"",""value"":1143}","{""axis"":{""x"":0,""y"":0,""z"":1},""fraction"":0.84486...","{""box_volume"":{""unit"":""mm3"",""value"":6241.536},...","{""multibody"":false,""patches"":{""count"":1,""not_t...","{""unit"":""mm3"",""value"":[374,4058,0,0,28,68,0]}",<NA>,"{""detected"":true,""positive_fraction_of_samples...","{""count"":0,""edge_list_url"":""s3://3dhubs-produc...",NaN,mm,completed,1187,0dd8b940-6b35-4ed5-a981-ce05e0734b4f
2,2022-02-16 16:27:18,2022-02-16 16:27:19,2022-02-16 16:27:18,"{""unit"":""milliseconds"",""value"":67}","[{""center"":{""x"":4.254160968e-24,""y"":8.73124980...","{""unit"":""milliseconds"",""value"":1161}","{""axis"":{""x"":0,""y"":1,""z"":0},""fraction"":1,""orig...","{""box_volume"":{""unit"":""mm3"",""value"":10455.28},...","{""multibody"":false,""patches"":{""count"":1,""not_t...","{""unit"":""mm3"",""value"":[591,2421,2404,1524,1009...",<NA>,"{""detected"":true,""positive_fraction_of_samples...","{""count"":0,""edge_list_url"":""s3://3dhubs-produc...",NaN,mm,completed,1228.0,9691095c-d562-4bda-813f-7dbe7398d686
3,2021-05-17 16:42:08,2021-05-17 16:42:10,2021-05-17 16:42:08,"{""unit"":""milliseconds"",""value"":57}","[{""center"":{""x"":0,""y"":0,""z"":28.8500003815},""di...","{""unit"":""milliseconds"",""value"":1910}","{""axis"":{""x"":0,""y"":0,""z"":1},""fraction"":0.88210...","{""box_volume"":{""unit"":""mm3"",""value"":26218.712}...","{""multibody"":false,""patches"":{""count"":1,""not_t...","{""unit"":""mm3"",""value"":[3588,8766,4560,516,780,...","[{""center"":{""x"":-2.782968637e-24,""y"":-2.782968...","{""detected"":false,""positive_fraction_of_sample...","{""count"":0,""edge_list_url"":""s3://3dhubs-produc...",NaN,mm,completed,1967,66102ac6-37a3-45b7-97dd-56d02b6ac651
4,2019-11-01 05:34:37,2019-11-01 05:34:43,2019-11-01 05:34:37,"{""unit"":""milliseconds"",""value"":75}",<NA>,"{""unit"":""milliseconds"",""value"":5949}","{""axis"":{""x"":1,""y"":0,""z"":0},""fraction"":0.93670...","{""box_volume"":{""unit"":""mm3"",""value"":184171.806...","{""multibody"":false,""patches"":{""count"":1,""not_t...","{""unit"":""mm3"",""value"":[91270,55336,40349,10925...",<NA>,<NA>,"{""count"":0,""edge_list_url"":""s3://3dhubs-produc...",NaN,mm,completed,6024,2c9d084c-dd69-4aff-8b85-a50d0f657372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2022-09-28 04:09:00,2022-09-28 04:09:09,2022-09-28 04:09:00,"{""unit"":""milliseconds"",""value"":139}","[{""center"":{""x"":9.0000002384,""y"":4.4444442689,...","{""unit"":""milliseconds"",""value"":8879}","{""axis"":{""x"":0,""y"":0,""z"":1},""fraction"":0.44566...","{""box_volume"":{""unit"":""mm3"",""value"":113319.36}...","{""multibody"":false,""patches"":{""count"":1,""not_t...","{""unit"":""mm3"",""value"":[85194,31491,6508,687,75...",<NA>,"{""detected"":false,""positive_fraction_of_sample...","{""count"":38,""edge_list_url"":""s3://3dhubs-produ...",NaN,mm,completed,9018.0,8de71a88-b70f-4597-b02a-c4527e0bc4e5
19996,2021-02-25 12:34:50,2021-10-18 20:18:42,2021-10-18 20:18:40,"{""unit"":""milliseconds"",""value"":81}","[{""center"":{""x"":-77.1317098612,""y"":-76.9670063...","{""unit"":""mil

Next, I check the data types of the columns in the DataFrame. Most of the columns are technically strings, but a lot of these columns contain deeply nested dictionaries/objects containing data on different things about the 3D model.

In [4]:
data.dtypes

created                  string
updated                  string
queued                   string
geometric_heuristics     string
holes                    string
job_run_time             string
latheability             string
machining_directions     string
multipart                string
neighbors                string
poles                    string
sheet_like_shape         string
unmachinable_edges       string
extrusion_height        float64
units                    string
status                   string
time                     string
uuid                     object
dtype: object

An example is the `holes` column. Let's pull up a random row with this field. This one has 8 holes with the following keys: `center`, `direction`, `end1`, `end2`, `facet_count`, `length`, and `radius`. Within the `center` and `direction` keys, there are these following keys: `x`, `y`, and `z`. Within `end1` and `end2`, there are these following keys: `closed` and `reachable`.

All this information is stored in only one column in the Parquet file. This is not analytics-friendly, and thus should be flattened. Additionally, some rows have `<NA>` values in the `holes` field. The holes data should be moved to another table as well.

In [5]:
json.loads(data.loc[8].holes)

[{'center': {'x': -90.9999647821, 'y': 10.75000067, 'z': 1.24627e-05},
  'direction': {'x': -0.0004851845, 'y': 0.9999910869, 'z': -0.0001722608},
  'end1': {'closed': False, 'reachable': True},
  'end2': {'closed': False, 'reachable': True},
  'facet_count': 137,
  'length': 20.0064318157,
  'radius': 6.2500018623},
 {'center': {'x': -45.4999648063, 'y': 10.7500006742, 'z': -78.8082992115},
  'direction': {'x': -0.0004851623, 'y': 0.9999910874, 'z': -0.0001721883},
  'end1': {'closed': False, 'reachable': True},
  'end2': {'closed': False, 'reachable': True},
  'facet_count': 137,
  'length': 20.0064312413,
  'radius': 6.2500017931},
 {'center': {'x': -45.4999648045, 'y': 10.7500006753, 'z': 78.8083242803},
  'direction': {'x': -0.0004851884, 'y': 0.9999910873, 'z': -0.0001721856},
  'end1': {'closed': False, 'reachable': True},
  'end2': {'closed': False, 'reachable': True},
  'facet_count': 137,
  'length': 20.0064315358,
  'radius': 6.2500017029},
 {'center': {'x': 3.95064e-05, 'y'

In [6]:
# Number of rows with <NA> holes value
len(data[pd.isna(data.holes)])

9789

Additionally, I noticed that the `facet_count` values in the `holes` column have different data

### Transform and load `holes` Pandas DataFrame to SQLite table

In [7]:
engine = create_engine("sqlite://", echo=False)

transformed_tables = main()
holes_df = transformed_tables["holes"]

# Import to the SQL DB all the transformed tables:
# NOTE: JSON/object data type is not supported in SQLite so I cast it to string data type
for k,v in transformed_tables.items():
    v.to_sql(k, con=engine, if_exists="append")

Transforming data...
Done! Elapsed time: 36.980764389038086


### Run queries

In [8]:
def run_query(query):
    results = []
    with engine.connect() as conn:
        results = conn.execute(text(query)).fetchall()

    conn.close()

    return results
    

Results obtained in **Python (Pandas field calculation)**:

In [9]:
# Has Unreachable Hole Warning

warning_query = """
    select count(distinct case when has_unreachable_hole_warning = True then uuid else null end),
        count(distinct case when has_unreachable_hole_warning = True then uuid else null end) * 100.0 / count(distinct uuid)
    from holes
"""
warning_results = run_query(warning_query)
print("The unique models that should trigger an unreachable hole warning are:", warning_results[0][0])
print("% Over Total UUID with holes:", str(round(warning_results[0][1], 2)) + "%")

The unique models that should trigger an unreachable hole warning are: 326
% Over Total UUID with holes: 3.19%


In [10]:
# Has Unreachable Hole Error

error_query = """
    select count(distinct case when has_unreachable_hole_error = True then uuid else null end),
        count(distinct case when has_unreachable_hole_error = True then uuid else null end) * 100.0 / count(distinct uuid)
    from holes
"""
error_results = run_query(error_query)
print("The unique models that should trigger an unreachable hole error are:", error_results[0][0])
print("% Over Total UUID with holes:", str(round(error_results[0][1], 2)) + "%")

The unique models that should trigger an unreachable hole error are: 39
% Over Total UUID with holes: 0.38%


With the flattened `holes` table loaded in SQL, I try to find a solution through **SQL** as well:

In [11]:
sql_has_unreachable_holes_warning_query = """
    with poor_ratio as (
        select uuid, (sum(distinct case when length > radius * 20 then 1 else 0 end) > 0) as has_poor_ratio
        from holes
        group by uuid
    )

    select count(distinct case when has_poor_ratio = 1 then uuid else null end),
        count(distinct case when has_poor_ratio = 1 then uuid else null end) * 100.0 / count(distinct uuid)
    from poor_ratio
"""
sql_has_unreachable_holes_warning_solution = run_query(sql_has_unreachable_holes_warning_query)
print("The unique models that should trigger an unreachable hole warning are:", sql_has_unreachable_holes_warning_solution[0][0])
print("% Over Total UUID with holes:", str(round(sql_has_unreachable_holes_warning_solution[0][1], 2)) + "%")

The unique models that should trigger an unreachable hole warning are: 326
% Over Total UUID with holes: 3.19%


In [12]:
sql_has_unreachable_holes_error_query = """
    with critical_ratio as (
        select uuid, (sum(distinct case when length > radius * 80 then 1 else 0 end) > 0) as has_critical_ratio
        from holes
        group by uuid
    )

    select count(distinct case when has_critical_ratio = 1 then uuid else null end),
        count(distinct case when has_critical_ratio = 1 then uuid else null end) * 100.0 / count(distinct uuid)
    from critical_ratio
"""
sql_has_unreachable_holes_error_solution = run_query(sql_has_unreachable_holes_error_query)
print("The unique models that should trigger an unreachable hole error are:", sql_has_unreachable_holes_error_solution[0][0])
print("% Over Total UUID with holes:", str(round(sql_has_unreachable_holes_error_solution[0][1], 2)) + "%")

The unique models that should trigger an unreachable hole error are: 39
% Over Total UUID with holes: 0.38%


The two solutions arrive at the same answers.

All models with a warning/error for unreachable holes are tagged as "completed":

In [13]:
# Models with Unreachable Hole Warning Count Grouped by Status

warning_status_query = """
    select status,
        count(distinct uuid)
    from models m
    where has_unreachable_hole_warning = True
    group by status
"""
warning_status_results = run_query(warning_status_query)

for row in warning_status_results:
    print(str(row[0]) + ":", row[1])

completed: 326


In [14]:
# Models with Unreachable Hole Error Count Grouped by Status

error_status_query = """
    select status,
        count(distinct uuid)
    from models m
    where has_unreachable_hole_error = True
    group by status
"""
error_status_results = run_query(error_status_query)

for row in error_status_results:
    print(str(row[0]) + ":", row[1])

completed: 39


Finally, going through the transformed tables in the SQL database, the size and the volume of the data in the given Parquet file can be seen more clearly, as they are flattened.

In [15]:
table_names = ["holes", "latheability", "machining_directions", "models", "neighbors", "poles"]

print("Number of rows per table:")
for t in table_names:
    print("- " + str(t) + ":", run_query("select count(*) from {}".format(t))[0][0])

Number of rows per table:
- holes: 135015
- latheability: 17503
- machining_directions: 19938
- models: 20000
- neighbors: 139566
- poles: 6307


If I could flatten the data more, I would most likely explode the `direction_removable_volume` and `selected_directions` fields in the `machining_directions` table, but for now I don't fully get the context of where this information is used, so I did not do that in this project.

Thanks for reading through my work!